In [1]:
import json


with open("data/raw/poetrydb_poems.json", "r") as f:
    poem_data = json.load(f)

In [2]:
# get a list of the poem lines
poems_and_info = poem_data['items']
print(type(poems_and_info[0]))
print(list(poems_and_info)[0])
poem_lines = [poem["lines"] for poem in poems_and_info]
# line_counts = [len(poem["lines"]) for poem in poems_and_info]
# line_counts[0:5]

<class 'dict'>
{'title': 'A Song of Autumn', 'author': 'Adam Lindsay Gordon', 'lines': ['‘WHERE shall we go for our garlands glad', 'At the falling of the year,', 'When the burnt-up banks are yellow and sad,', 'When the boughs are yellow and sere?', 'Where are the old ones that once we had,', 'And when are the new ones near?', 'What shall we do for our garlands glad', 'At the falling of the year?’', '‘Child! can I tell where the garlands go?', 'Can I say where the lost leaves veer', 'On the brown-burnt banks, when the wild winds blow,', 'When they drift through the dead-wood drear?', 'Girl! when the garlands of next year glow,', 'You may gather again, my dear—', 'But I go where the last year’s lost leaves go', 'At the falling of the year.’'], 'linecount': '16'}


In [44]:
import re
from functools import reduce


# pattern to remove any punctuation
punctuation_pattern = r"[^A-Za-z0-9\s]"


cleaned_poem_lines = []


for poem in poem_lines:   # or poems_and_info, depending on your variable name
  


   processed_lines = []
  
   for line in poem:
       # remove punctuation
       line = re.sub(punctuation_pattern, "", line)


       # collapse multiple spaces → one
       line = re.sub(r"\s+", " ", line).strip().lower()


       # skip empty lines
       if line:
           processed_lines.append(line)


   cleaned_poem_lines.append(processed_lines)


# check
for i, song in enumerate(cleaned_poem_lines[0:3]):
    print(song[0])
    print()


where shall we go for our garlands glad

the ocean heaves around us still

i have a rendezvous with death



In [4]:
# get mean words per line and number of lines for each poem
mean_words_per_poem = []
num_lines_per_poem = []

for poem in poem_lines:
    num_lines = len(poem)
    num_lines_per_poem.append(num_lines)

    if num_lines == 0:
        mean_words_per_poem.append(0)
        print("Cleaned poem lyrics was an empty string, inspect what caused this")
        continue

    word_counts = [len(line.split()) for line in poem]
    
    mean_words = sum(word_counts) / num_lines
    mean_words_per_poem.append(mean_words)

In [9]:
# before you run this you need to run "!pip install pronouncing" in terminal
import pronouncing


rhyme_feature_poems = []


# how many of the future lines we compare with
lines_to_check = 4


for poem in cleaned_poem_lines:
  
   # grabs the last word of each line in the poem 
   # (lines already cleaned earlier in preprocessing)
   last_words = []
   for line in poem:
       words = line.split()
       last_words.append(words[-1].lower())
  
   # no rhyme density if we only have one line (or none)
   if len(last_words) < 2:
       rhyme_feature_poems.append(0)
       continue

   rhyme_counts = 0
   words_checked = 0

   # counts rhymes up to 3 lines appart
   for i in range(len(last_words)):
       for offset in range(1, lines_to_check + 1):
           if i + offset >= len(last_words):
               break

           w1 = last_words[i]
           w2 = last_words[i + offset]
           words_checked += 1

           # using CMUdict to return how the word sounds
           phones1 = pronouncing.phones_for_word(w1)
           phones2 = pronouncing.phones_for_word(w2)


           if phones1 and phones2:
               # returns set of words that rhyme with the 
               # last word in the first line
               rhymes1 = pronouncing.rhymes(w1)
              
               # checks if the last word in the comparison line
               # is in that set
               if w2 in rhymes1:
                   rhyme_counts += 1
           else:
               # for slang, invented words, missing CMUdict entries, etc
               # estimate rhyme manually by checking last 2 letters
               if w1[-2:] == w2[-2:]:
                   rhyme_counts += 1


   # what percentage of all checked pairs rhyme
   rhyme_density_value = rhyme_counts / words_checked
   rhyme_feature_poems.append(rhyme_density_value)


rhyme_feature_poems[:20]


[0.4074074074074074,
 0.1588785046728972,
 0.10465116279069768,
 0.11083743842364532,
 0.09855072463768116,
 0.17169811320754716,
 0.06268221574344024,
 0.046583850931677016,
 0.1111111111111111,
 0.11403508771929824,
 0.22380952380952382,
 0.14814814814814814,
 0.11557788944723618,
 0.11197754210854648,
 0.05263157894736842,
 0.08139534883720931,
 0.11538461538461539,
 0.11538461538461539,
 0.14084507042253522,
 0.11216730038022814]

In [10]:
# checks - check these are the correct shape

print(len(cleaned_poem_lines))
print(len(cleaned_poem_lines) == len(mean_words_per_poem) &
     len(mean_words_per_poem) == len(num_lines_per_poem) & 
     len(num_lines_per_poem) == len(rhyme_feature_poems))

3413
True


In [11]:
with open("data/processed/combined_songs_large_fixed.json", "r") as f:
        song_data = json.load(f)

In [12]:
import re
from functools import reduce

songs_and_info = song_data['items']
# one song dictionary
print(songs_and_info[0])
# Pulling each song's lyrics
# Note: each item also contains title, album and duration if we want that later

raw_song_texts = [song["lyrics"] for song in songs_and_info]
raw_song_texts[0:5]


{'title': 'Father Figure', 'artist': 'Taylor Swift', 'spotify_artist_name': 'Taylor Swift', 'album': 'The Life of a Showgirl', 'release_date': '2025-10-03', 'duration_ms': 212777, 'popularity': 94, 'lyrics': '[Verse 1]\nWhen I found you, you were young, wayward, lost in the cold\nPulled up to you in the Jag\', turned your rags into gold\nThe winding road leads to the chateau\n"You remind me of a younger me," I saw potential\n[Chorus]\nI\'ll be your father figure,\n \nI drink that brown liquor\nI can make deals with the devil because my dick\'s bigger\nThis love is pure profit, just step into my office\nI dry your tears with my sleeve\n[Post-Chorus]\nLeave it with me, I protect the family\nLeave it with me, I protect the family\n[Verse 2]\nI pay the check before it kisses the mahogany grain\nSaid, "They wanna see you rise, they don\'t want you to reign"\nI showed you all the tricks of the trade\nAll I asked for is your loyalty, my dear protégé\n[Chorus]\nI\'ll be your father figure,\n I

['[Verse 1]\nWhen I found you, you were young, wayward, lost in the cold\nPulled up to you in the Jag\', turned your rags into gold\nThe winding road leads to the chateau\n"You remind me of a younger me," I saw potential\n[Chorus]\nI\'ll be your father figure,\n \nI drink that brown liquor\nI can make deals with the devil because my dick\'s bigger\nThis love is pure profit, just step into my office\nI dry your tears with my sleeve\n[Post-Chorus]\nLeave it with me, I protect the family\nLeave it with me, I protect the family\n[Verse 2]\nI pay the check before it kisses the mahogany grain\nSaid, "They wanna see you rise, they don\'t want you to reign"\nI showed you all the tricks of the trade\nAll I asked for is your loyalty, my dear protégé\n[Chorus]\nI\'ll be your father figure,\n I drink that brown liquor\nI can make deals with the devil because my dick\'s bigger\nThis love is pure profit, just step into my office\nThey\'ll know your name in the streets\n\n[Post-Chorus]\nLeave it with

In [40]:
# when there are labels (verse, chorus, etc) 
# in between two enters remove them so we don't count the label 
# as an extra line 
cleaned_songs = [
    re.sub(r"\[(Verse \d+|Chorus|Intro|Bridge|Pre-Chorus)[^\]]*\]", "", song)
    for song in raw_song_texts
]
for i, song in enumerate(cleaned_songs[0:3]):
    print(song[:100])
    print()



When I found you, you were young, wayward, lost in the cold
Pulled up to you in the Jag', turned yo


I had a bad habit
Of missing lovers past
My brother used to call it
"Eating out of the trash"
It's 


Elizabeth Taylor
Do you think it's forever?

That view of Portofino was on my mind when you called 



In [42]:
# remove verse/chorus labels, newline characters and punctuations
replacements_space = [
    r"\[Verse \d+\]",
    r"\[Chorus\]",
    r"\r?\n",
]

punctuation_pattern = r"[^A-Za-z0-9\s]"

cleaned_song_lines = []

for song in cleaned_songs:
    # there are some observations labeled as lyrics that are not songs
    #  and contain other info, and they begin with this
    if "Contributors\nTranslations" in song:
        continue

    
    lines = song.split("\n")
    processed_lines = []
    
    for line in lines:
        # remove labels and punctuation
        line = reduce(lambda text, pat: re.sub(pat, " ", text), replacements_space, line)
        line = re.sub(punctuation_pattern, "", line)

        # turns multiple spaces into one
        line = re.sub(r"\s+", " ", line).strip().lower()

        # removes any lines that are empty or only contain whitespace
        if line:
            processed_lines.append(line)

    cleaned_song_lines.append(processed_lines)

# checking we removed all correctly
for i, song in enumerate(cleaned_song_lines[20:27]):
    print(song[0:2])



['yeah', 'girl i cant decide if i should stay and say i tried']
['ride with me', 'ride with me']
['young money', 'yeah four']
['okay well', 'oh']
['ayy', 'can i trust you']
['yeah of course of course i got feelings for you but', 'forgot']
['mothballs i cant get the smell out my clothes', 'yyz racing cause the gate bout to close']


In [15]:
# get mean words per line and number of lines for each song
mean_words_per_song = []
num_lines_per_song = []
total_words_per_song = []

for song in cleaned_song_lines:

    num_lines = len(song)
    num_lines_per_song.append(num_lines)

    if num_lines == 0:
        mean_words_per_song.append(0)
        print("Cleaned songs lyrics was an empty string, inspect what caused this")
        continue

    word_counts = [len(line.split()) for line in song]

    mean_words = sum(word_counts) / num_lines
    mean_words_per_song.append(mean_words)
    total_words_per_song.append(sum(word_counts))


In [16]:
# calculating mean WPM for each song
durations_ms = [song["duration_ms"] for song in songs_and_info]
durations_min = [song["duration_ms"] / 60000 for song in songs_and_info]

wpm_per_song = []

for words, minutes in zip(total_words_per_song, durations_min):
    if minutes == 0:
        wpm_per_song.append(0)
        print("If we have a song with no time length, this is an error and we set WPM to 0")
    else:
        wpm_per_song.append(words / minutes)

In [17]:
# before you run this you need to run "!pip install pronouncing" in terminal
import pronouncing


rhyme_feature_songs = []
# no other changes please

# how many of the future lines we compare with
lines_to_check = 4


for song in cleaned_song_lines:
  
    # grabs the last word of each line in the song 
    # (lines already cleaned earlier in preprocessing)
    last_words = []
    for line in song:
        words = line.split()
        last_words.append(words[-1].lower())
  
    # no rhyme density if we only have one line (or none)
    if len(last_words) < 2:
        rhyme_feature_songs.append(0)
        continue

    rhyme_counts = 0
    words_checked = 0

    # counts rhymes up to 3 lines apart
    for i in range(len(last_words)):
        for offset in range(1, lines_to_check + 1):
            if i + offset >= len(last_words):
                break

            w1 = last_words[i]
            w2 = last_words[i + offset]
            words_checked += 1

            # using CMUdict to return how the word sounds
            phones1 = pronouncing.phones_for_word(w1)
            phones2 = pronouncing.phones_for_word(w2)

            if phones1 and phones2:
                # returns set of words that rhyme with the 
                # last word in the first line
                rhymes1 = pronouncing.rhymes(w1)
              
                # checks if the last word in the comparison line
                # is in that set
                if w2 in rhymes1:
                    rhyme_counts += 1
            else:
                # for slang, invented words, missing CMUdict entries, etc
                # estimate rhyme manually by checking last 2 letters
                if w1[-2:] == w2[-2:]:
                    rhyme_counts += 1

    # what percentage of all checked pairs rhyme
    rhyme_density_value = rhyme_counts / words_checked
    rhyme_feature_songs.append(rhyme_density_value)


rhyme_feature_songs[:20]


[0.030927835051546393,
 0.032,
 0.03383458646616541,
 0.11139896373056994,
 0.010067114093959731,
 0.018072289156626505,
 0.024271844660194174,
 0.02912621359223301,
 0.02032520325203252,
 0.03896103896103896,
 0.02702702702702703,
 0.0,
 0.06310679611650485,
 0.00967741935483871,
 0.024390243902439025,
 0.02066115702479339,
 0.019801980198019802,
 0.04242424242424243,
 0.021739130434782608,
 0.029411764705882353]

In [20]:
# checks
print(len(cleaned_songs))
print(len(cleaned_songs) == len(mean_words_per_song) &
     len(mean_words_per_song) == len(wpm_per_song) & 
     len(wpm_per_song) == len(num_lines_per_song) & 
     len(num_lines_per_song) == len(rhyme_feature_songs))

3000
True


In [21]:
import numpy as np

# combining these calculated features into a np array before standardizing 
features_array_poems_unstandardized = np.column_stack([
    mean_words_per_poem,
    num_lines_per_poem,
    rhyme_feature_poems
])

features_array_songs_unstandardized = np.column_stack([
    mean_words_per_song,
    num_lines_per_song,
    wpm_per_song,
    rhyme_feature_songs
])
print(features_array_poems_unstandardized.shape)
print(features_array_songs_unstandardized.shape)

(3413, 3)
(3000, 4)


In [22]:
# standardizing
poem_means = features_array_poems_unstandardized.mean(axis=0)
poem_stds  = features_array_poems_unstandardized.std(axis=0)

poem_stds[poem_stds == 0] = 1

features_array_poems = (features_array_poems_unstandardized - poem_means) / poem_stds

song_means = features_array_songs_unstandardized.mean(axis=0)
song_stds  = features_array_songs_unstandardized.std(axis=0)

song_stds[song_stds == 0] = 1

features_array_songs = (features_array_songs_unstandardized - song_means) / song_stds